<a href="https://colab.research.google.com/github/tiagopessoalima/ED2/blob/main/Semana_14_(ED2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **B-Trees e Heaps**

Este material aborda duas estruturas de dados essenciais para otimizar operações de acesso, armazenamento e ordenação: B-Trees (árvores balanceadas de múltiplos caminhos) e Heaps (estruturas hierárquicas com propriedade de ordenação parcial). Ambas são amplamente empregadas em bancos de dados, sistemas de arquivos, algoritmos de agendamento e estruturas de prioridade.

## **B-Trees**

Uma B-Tree é uma estrutura de dados em forma de árvore que permite operações de busca, inserção e exclusão em tempo logarítmico. Ela é projetada para ser eficiente em sistemas que armazenam dados em dispositivos de armazenamento secundário, como discos rígidos, onde o acesso a blocos de dados é mais lento.



## **Propriedades de uma B-Tree**

- Cada nó pode ter no máximo $m$ filhos, onde $m$ é a ordem da árvore.
- Cada nó, exceto a raiz, deve ter pelo menos $\lceil m/2 \rceil$ filhos.
- Todos os nós folha estão no mesmo nível, garantindo que a árvore seja balanceada.
- Um nó com $k$ filhos contém $k-1$ chaves.

Aqui está um exemplo visual de uma B-Tree de ordem 3 (m=3) que ilustra todas as propriedades mencionadas:

```
        [40]                 ← Raiz
      /      \
  [20]         [60,70]       ← Nó intermediário com 2 chaves
 /    \       /   |   \
[10] [30]  [50]  [65]  [80]  ← Folhas (mesmo nível)
```

**Análise das propriedades:**

1. Máximo de filhos ($m$):

| Nó         | Filhos | Status (≤ 3) |
|------------|--------|--------------|
| Raiz [40]  | 2      | ✓            |
| [20]       | 2      | ✓            |
| [60,70]    | 3      | ✓            |

2. Mínimo de filhos (exceto raiz): $⌈m/2⌉ = ⌈3/2⌉ = 2$

| Nó       | Filhos | Status |
|----------|--------|--------|
| [20]     | 2      | ✓      |
| [60,70]  | 3      | ✓      |

3. Folhas no mesmo nível
- Todas as folhas ([10], [30], [50], [65], [80]) estão no nível 2 → ✓
- Distribuição por nível:
 - Nível 0: [40] (Raíz)
 - Nível 1: [20], [60,70] (Nós intermediários)
 - Nível 2: [10], [30], [50], [65], [80] (Folhas)

4. Relação chaves-filhos ($k$ filhos → $k-1$ chaves)

| Nó        | Filhos (k) | Chaves Esperadas (k−1) | Chaves Reais     | Status |
|-----------|-------------|-------------------------|------------------|--------|
| [40]      | 2           | 1                       | [40] (1)         | ✓      |
| [20]      | 2           | 1                       | [20] (1)         | ✓      |
| [60,70]   | 3           | 2                       | [60,70] (2)      | ✓      |






##**Operações em B-Trees**

### **Busca**

A operação de busca em uma B-Tree inicia na raiz e percorre recursivamente a subárvore correspondente ao intervalo onde a chave pode estar. A cada nó visitado, realiza-se uma busca binária (ou linear, em implementações simples) entre as chaves armazenadas para decidir qual ponteiro de filho seguir. O processo continua até que a chave seja encontrada ou se determine que ela não está presente na árvore.

#### **Exemplo de Busca em uma B-Tree (Ordem 3)**

```
        [40]
       /    \
   [20]      [60,70]
   /  \     /   |   \
 [10] [30] [50] [65] [80]
```

##### **Caso 1: Busca pelo valor 50 (existente)**



1. Raiz [40]:
- 50 > 40 → Vá para o filho direito (subárvore direita)
- Acesso ao nó: [60,70]

2. Nó [60,70]:
- Compare 50 com as chaves:
- 50 < 60 → Vá para o primeiro filho (à esquerda de 60)
- Acesso ao nó: [50]

3. Nó folha [50]:
- Chave 50 encontrada!
- Busca bem-sucedida

Caminho percorrido: `[40] → [60,70] → [50]`

##### **Caso 2: Busca pelo valor 65 (existente)**



1. Raiz [40]:
- 65 > 40 → Vá para o filho direito
- Acesso ao nó: [60,70]

2. Nó [60,70]:
- Compare 65 com as chaves:
 - 60 ≤ 65 < 70 → Vá para o segundo filho (entre 60 e 70)
- Acesso ao nó: [65]

3. Nó folha [65]:
- Chave 65 encontrada!
- Busca bem-sucedida

Caminho percorrido: `[40] → [60,70] → [65]`

##### **Caso 3: Busca pelo valor 35 (inexistente)**



1. Raiz [40]:
- 35 < 40 → Vá para o filho esquerdo
- Acesso ao nó: [20]

2. Nó [20]:
- Compare 35 com as chaves:
 - 35 > 20 → Vá para o segundo filho (à direita de 20)
- Acesso ao nó: [30]

3. Nó folha [30]:
- Chaves: 30 < 35 → Não encontrado
- Busca mal-sucedida

Caminho percorrido: `[40] → [20] → [30]`

##### **Caso 4: Busca pelo valor 70 (no nó interno)**

1. Raiz [40]:
- 70 > 40 → Vá para o filho direito
- Acesso ao nó: [60,70]

2. Nó [60,70]:
- Chave 70 encontrada diretamente no nó!
- Busca bem-sucedida sem acessar folhas

Caminho percorrido: `[40] → [60,70]`

#### **Complexidade da Busca**

- Altura da árvore: $O(\log_m n)$
- Custo por nó: $O(\log m)$ (busca binária nas chaves)
- Custo total: $O(\log_m n * \log m) = O(\log n)$
- Vantagem principal: Minimiza acessos a disco (cada nó equivale a uma página de disco)

### **Inserção**

A operação mantém o balanceamento através de divisões estratégicas de nós. O processo ocorre em três etapas principais:
1. Localizar a folha correta usando busca
2. Inserir ordenadamente na folha
3. Tratar **overflow** com divisão recursiva (split)

> **Overflow** refere-se a uma condição em que um nó excede sua capacidade máxima de armazenamento de chaves.

### **Exemplo de Inserção em B-Tree (m=3, máximo 2 chaves/nó e mínimo de 1 chave/nó)**


```
        [40]
       /    \
   [20]     [60]
  /   \     /   \
[10]  [30][50]  [70]
```

#### **Caso 1: Inserir 80**


1. **Localizar folha:** [40] → [60] → [70]
2. **Inserir ordenado:** [70,80] ✓

**Resultado:**

```
        [40]
       /    \
   [20]     [60]
  /   \     /   \
[10] [30] [50] [70,80]
```

#### **Caso 2: Inserir 65**


1. **Localizar folha:** [40] → [60] → [70,80]
2. **Inserir ordenado:** [65,70,80] → OVERFLOW (3 chaves > máximo 2)
3. **Divisão (split):**
- Chave mediana (70) sobe para o pai [60]
- Criar novo nó direito [80]
- Nó original vira [65]
4. **Atualizar pai:** [60,70]

**Resultado:**

```
        [40]
       /    \
   [20]     [60,70]  
  /   \    /   |   \
[10][30] [50] [65] [80]
```


#### **Caso 3: Inserir 55**


1. **Localizar folha:** [40] → [60,70] → [50]
2. **Inserir ordenado:** [50,55] ✓

```
        [40]
       /     \
   [20]        [60,70]
  /   \       /   |   \
[10] [30] [50,55][65][80]
```

#### **Caso 4: Inserir 58**

1. **Localizar folha:** [40] → [60,70] → [50,55]
2. **Inserir ordenado:** [50,55,58] → *OVERFLOW* (3 chaves > máximo 2)
3. **Divisão (split):**
- Chave mediana (55) sobe para o pai [60,70]
- Criar novo nó direito [58]
- Nó original vira [50]

**Estado Intermediário**

```
        [40]
       /    \
   [20]     [55,60,70]   ✗ (OVERFLOW)
  /   \    /   /   \   \
[10][30] [50] [58][65][80]
```

4. **Divisão do Nó Interno (Propagação):**
- Chave mediana (60) sobe para o pai [40]
- Criar novos nós intermediários:
 - Esquerda: [55] (herda filhos [50] e [58])
 - Direita: [70] (herda filhos [65] e [80])

**Árvore resultante:**

```
          [40,60]        
        /    |    \
     [20]   [55]   [70]   
    /   \   /   \   /  \
 [10] [30][50] [58][65][80]
```

#### **Complexidade da Inserção**


- Busca da folha: $O(\log_m n)$ níveis × $O(\log m)$ por nível = $O(\log n)$
- Divisões: Até $O(\log_m n)$ divisões, cada uma custando $O(m)$
- Custo total: $O(m \cdot \log_m n) = O(\log n)$
- Eficiência prática: A maioria das inserções não causa divisões propagadas

### **Exclusão**

A operação de exclusão em uma B-Tree remove uma chave enquanto mantém as propriedades da árvore, como o balanceamento e o número mínimo de chaves por nó. A exclusão é mais complexa que a inserção, pois pode causar *underflow* (um nó com menos chaves do que o mínimo permitido). Para lidar com isso, a B-Tree usa estratégias como redistribuição de chaves ou fusão de nós.

**Passos Gerais da Exclusão:**

1. **Localizar a chave:** Usar a busca para encontrar o nó que contém a chave a ser removida.

2. **Remover a chave:**
- Se a chave está em um nó folha, removê-la diretamente.
- Se a chave está em um nó interno, substituí-la pela chave predecessora (maior chave da subárvore esquerda) ou sucessora (menor chave da subárvore direita) e remover essa chave da folha.

3. **Corrigir underflow:**
- Se o nó ficar com menos de $\lceil m/2 \rceil - 1$ chaves (para ordem  $m$), realizar redistribuição (borrow) de um nó irmão ou fusão (merge) com um irmão.
- Propagar ajustes para os nós pais, se necessário.

### **Exemplo de Eclusão em B-Tree (m=3, máximo 2 chaves/nó e mínimo de 1 chave/nó)**


```
          [40,60]
        /    |    \
     [20]   [55]   [70]
    /   \   /   \   /  \
 [10] [30][50] [58][65][80,81]
```

#### **Caso 1: Remover 81**

1. **Localizar a chave:**
- Iniciar na raiz [40,60].
- 81 > 60 → Ir para o filho à direita de 60: [70].
- 81 > 70 → Ir para o filho à direita de 70: [80,81].
- Chave 81 encontrada no nó folha [80,81].

2. **Remover a chave:**
- Remover 81 do nó [80,81], resultando em [80].
- Verificar o número de chaves: [80] tem 1 chave, que é igual ao mínimo (⌈m/2⌉ - 1 = ⌈3/2⌉ - 1 = 1). Não há *underflow*.

**Árvore resultante:**

```
          [40,60]        
        /    |    \
     [20]   [55]   [70]   
    /   \   /   \   /  \
 [10] [30][50] [58][65][80]
```


#### **Caso 2: Remover 58**

1. **Localizar a chave:**
- Raiz [40,60]: 58 > 40 e 58 < 60 → Ir para o filho entre 40 e 60: [55].
- Nó [55]: 58 > 55 → Ir para o filho à direita de 55: [58].
- Chave 58 encontrada no nó folha [58].

2. **Remover a chave:**
- Remover 58 do nó [58], resultando em [] (0 chaves).
- Verificar: 0 chaves < 1 (mínimo) → *Underflow*.

3. **Corrigir underflow:**
- **Redistribuição (borrow):**
 - Irmão adjacente esquerdo: [50] (sob o mesmo pai [55])
 - [50] não pode emprestar (única chave = mínimo) ✗
- **Fusão (merge):**
 - Fundir nó vazio com irmão esquerdo [50] + chave do pai [55] → Novo nó: [50,55]
 - Pai [55] perde chave e filho → Pai vazio → Underflow propagado!
4. **Corrigir underflow no pai ([] vazio):**
 - Fusão com irmão + chave da raiz:
    - Fundir pai vazio ([55]) + irmão esquerdo ([20]) + chave intermediária da raiz (40)
    - Novo nó: [20,40] (herda filhos de [20] e [50,55])
    - Raiz atualizada: [60] (única chave restante)

**Árvore resultante:**

```
        [60]                
       /     \
  [20,40]     [70]          
 /    |   \     /  \
[10][30][50,55][65][80]
```

#### **Caso 3: Remover 60**

1. **Localizar a chave:**
- 60 está na raiz (nó interno).

2. **Substituir pela predecessora:**
- Predecessora de 60: Maior chave na subárvore esquerda (folha direita de [20,40] → [50,55]).
- Predecessora = 55.

**Estado Intermediário**

```
        [55]       
       /     \
  [20,40]     [70]          
 /    |   \     /  \
[10][30][50,55][65][80]
```

3. **Remover 55 da folha:**
- Folha original: [50,55] → após remoção: [50].
- Verificar *underflow*: [50] tem 1 chave (mínimo = ⌈3/2⌉ - 1 = 1) → OK!

**Árvore resultante:**

```
           [55]                 
         /     \
   [20,40]      [70]          
  /   |   \     /  \
 [10][30][50] [65][80]
```

### **Exercícios**

1. Considere uma B-Tree de ordem 3 inicialmente vazia. Realize as inserções na ordem: [40, 20, 60, 10, 30, 50, 70, 80, 65, 55, 58]

 a) Desenhe a árvore após cada inserção que causa overflow.

 b) Mostre o estado final da árvore.

2. Dada a B-Tree de ordem 3:

```
        [55]  
       /    \  
  [20,40]   [70]  
  / | \     /   \  
[10][30][50][65][80]  
```
 Mostre o caminho percorrido para buscar:

 a) Chave 65

 b) Chave 25

 c) Chave 40


3. Partindo da árvore do Exercício 2. Remova sequencialmente, mostrando a árvore após cada remoção e explique se houve underflow.

 a) 80

 b) 50

 c) 10

4. Partindo da árvore do Exercício 2. Remova sequencialmente, mostrando a passo a passo as fusões necessárias.

 a) 30

 b) 40

 c) 20

5. Considere a B-Tree de ordem 4:

```
         [30]  
       /      \  
  [10,20]      [50,60,70]  
 /   |   \    /   |   |   \  
[5][15][25][40][55][65][75][80]  
```

Remova:

a) 5 (use redistribuição com o irmão direito)

b) 75 (use redistribuição com o irmão esquerdo)

Desenhe a árvore após cada operação.

6. Construa uma B-Tree de ordem 4 com as chaves: [25, 10, 40, 15, 35, 5, 20, 50, 30, 45, 60, 55, 70]. Em seguida, remova: [40, 15, 25, 10]. Documente todas as divisões e fusões durante o processo.

7. Verifique se a estrutura abaixo é uma B-Tree válida de ordem 4:
```
           [22, 44]  
         /     |     \  
  [5,10]   [30,33]   [50,55,66]  
  /  |  \    /   | \    / | |  \  
[1][7][12][28][32][40][48][52][60][70]  
```
Justifique com base em:

 a) Número máximo de chaves por nó

 b) Número mínimo de filhos em nós internos

 c) Nível das folhas

 d) Relação chaves-filhos

## **O que são Heaps?**

Um Heap é uma árvore binária quase completa que satisfaz a propriedade de heap: em um max-heap, o valor de cada nó é maior ou igual ao de seus filhos; em um min-heap, é menor ou igual. Heaps são implementados eficientemente em arrays, com as seguintes características:

- Acessar o elemento de maior/menor prioridade (raiz) é ($O(1)$).
- Inserção e remoção têm complexidade ($O(\log n)$), onde ($n$) é o número de elementos.
- Usado em filas de prioridade, escalonamento de processos e no algoritmo Heap Sort.

Exemplo de Aplicação: Em sistemas operacionais, filas de prioridade baseadas em min-heaps gerenciam tarefas com base em sua urgência. O Heap Sort utiliza max-heaps para ordenar dados com complexidade ($O(n \log n)$).

Exemplo Visual (Max-Heap):

```
       90
      /  \
    70    80
   /  \   /
  50  60 20
```